In [2]:
import xml.sax
import sys
sys.path.append('/content/drive/My Drive/IRE')
import collections
import os
import os.path
import re
from collections import defaultdict
import Stemmer
# from nltk.stem.snowball import SnowballStemmer
# from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
import timeit
from parsedata import Parser
from bisect import bisect
from math import log10
from operator import itemgetter

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [1]:
!pip3 install PyStemmer
!pip3 install nltk

     |████████████████████████████████| 563kB 2.9MB/s 
  Created wheel for PyStemmer: filename=PyStemmer-2.0.1-cp36-cp36m-linux_x86_64.whl size=423789 sha256=8c9a53445a60a827f792eb5a3e60dae5d40e63d640bfc0c34c7fbe1d41c7ca93
  Stored in directory: /root/.cache/pip/wheels/f3/3c/11/ee323a09706e17a649c2730bd8819b06e887411ff7507acf7a
Successfully built PyStemmer


In [10]:
index_file_path = ""
query_file_path = ""
output_file_path = ""
total_docs = 0
id_title = dict()
secondary_index = []
queries = []
outputs = []
query_result = dict()

def process_title_file(id_title_file_path):
  global total_docs, id_title
  with open(id_title_file_path, 'r') as inf:
    for l in inf:
      id, title = l.split(":",1)
      id_title[id] = title
      total_docs += 1

def read_secondary_index(secondary_index_file_path):
  global secondary_index
  with open(secondary_index_file_path, 'r') as inf:
    for l in inf:
      secondary_index.append(l.split(":")[0])

def fetch_data(check_match, data_in_file):
  start_in = check_match.start()
  find_in = start_in+1
  end_in = data_in_file.find('\n', find_in)
  get_data = data_in_file[start_in:end_in]
  return get_data.split(':')[1].split('|')

def read_primary_index_file(filename, word):
  primary_file = open(filename, 'r')
  data_in_file = primary_file.read()
  check_match = re.search('^'+word+':', data_in_file, re.M)
  if check_match:
    fetched_data = fetch_data(check_match, data_in_file)    ##get posting list for that word from the file (list of docs#t2+i4)
    return fetched_data
  else:
    return []


def get_index_field_query(tag, word):
  global secondary_index, query_result, index_file_path, total_docs
  # print(word)
  # print(secondary_index)
  # print("word processed ",tag, word)
  indx = bisect(secondary_index, word)          ## search for the word in secondary list
  entire_list = list()
  if indx > 0:
    entire_list = read_primary_index_file(index_file_path+'primaryfile_'+str(indx)+'.txt', word)    ## get positing list corresponding 
                                                                                                   ## to that word from primary file
  df = len(entire_list)
  idf = log10(total_docs/(1+df))          ##cal idf score

  for lt in entire_list:
    wt_freq = 0
    docid, posting_list = lt.split('#')
    docid = docid[1:]
    for cat in posting_list.split('+'): #get all tags for that doc
      post_tag = cat[0]
      freq = int(cat[1:])
      if post_tag == tag:               ## if queried tag is same as posting tag
        wt_freq += freq*10000         
      else:
        wt_freq += freq

    tfidf = float(log10(1+wt_freq)*float(idf))        ## cal tfidf score
    if docid not in query_result:
      query_result[docid] = tfidf
    else:
      query_result[docid] += tfidf                    ## update tfidf score for the docid
    # print(docid, query_result[docid])
         


def process_field_query(tag, words):
  ex = re.compile(r'[\_]', re.DOTALL)
  words_ex = ex.sub(' ', words)
  words_list = re.findall("\d+|[\w]+", str(words_ex))     ##cleaning
  words_list = clean_words(words_list)
  for w in words_list:                          ## process for each word corresponding to queried tag
    get_index_field_query(tag, w.casefold())

def get_k_documents(k):
  global query_result, id_title, outputs
  cnt = 0
  dic_rev_sorted = dict(sorted(query_result.items(), key=itemgetter(1), reverse=True))  ## sort dict in reverse order of tfidf
 

  for doc,title in dic_rev_sorted.items():          ## get k doc ids and their title
    outputs.append(doc+', '+id_title[doc])
    cnt += 1
    if cnt == k:
      break
  query_result.clear()

def process_multi_field_query(query):
  words = query.split(' ')
  local_dict = {}
  for i in words:
    if ':' in i:
      tag, w = i.split(':')
      local_dict[tag] = [w.casefold().strip()]
    else:
      local_dict[tag].append(i.casefold())
  return local_dict

def clean_words(words):
  #words = words.casefold()
  stop_words = set(stopwords.words('english'))
  tokens = [t for t in words if t not in stop_words]
  stemmer = Stemmer.Stemmer('english')
  tokens = [stemmer.stemWord(w) for w in tokens]
  return tokens

def get_index_normal_query(word):
  indx = bisect(secondary_index, word)              ##search for word in secondary
  entire_list = list()
  if indx > 0:
    entire_list = read_primary_index_file(index_file_path+'primaryfile_'+str(indx)+'.txt', word)    ## get positng list
  
  df = len(entire_list)
  idf = log10(total_docs/(1+df))          ## cal idf

  li = {'t':100000, 'i':1, 'b':0.5, 'c':1, 'l':1, 'r':1}

  for lt in entire_list:
    wt_freq = 0
    docid, posting_list = lt.split('#')
    docid = docid[1:]
    # if docid == '3433930':
    #   print(posting_list)
    for cat in posting_list.split('+'):
      post_tag = cat[0]
      freq = int(cat[1:])
      wt_freq += li[post_tag]*freq                ## weighted freq

    tfidf = float(log10(1+wt_freq)*float(idf))    ## cal tfidf
    if docid not in query_result:
      query_result[docid] = tfidf
    else:
      query_result[docid] += tfidf          ## update tfidf score
    # if word == '2019':
    #   print(docid, query_result[docid])


def process_normal_query(words):
  words = words.lower()
  ex = re.compile(r'[\_]', re.DOTALL)
  words_ex = ex.sub(' ', words)
  words_list = re.findall("\d+|[\w]+", str(words_ex))
  words_list = clean_words(words_list)
  # print(words_list)
  for word in words_list:                 ## for each word in normal query
    get_index_normal_query(word)
    # print("processed ", word)
    # print(query_result)
    
  
  
def search_queries():
  global outputs, queries
  li = ['t:', 'i:', 'b:', 'c:', 'l:', 'r:']
  for q in queries:
      # outputs.append(q)
      print(q)
      #get count of req docs
      num,qu = q.split(',',1)
      num = int(num)
      qu = qu.strip()
      start = timeit.default_timer()
      ## check for field query
      if any(val in qu for val in li):
        processed_dictionary = process_multi_field_query(qu)  ## get words for each tag
        for tag,words in processed_dictionary.items():  ## process for each tag
          # print(tag, words)
          process_field_query(tag, ' '.join(words))
          # print("tfidf score after ", tag)
          # for k,v in query_result.items():
          #   print(k,v)
        get_k_documents(num)
        # print(outputs)
      else:
        process_normal_query(qu)
        get_k_documents(num)
        # print(outputs)
      stop = timeit.default_timer()
      outputs.append(str(stop-start)+' '+str((stop-start)/num)+'\n')
      print(str(stop-start)+' '+str((stop-start)/num)+'\n')
      outputs.append('\n')

if __name__=="__main__":
  global index_file_path, query_file_path, output_file_path, queries 
  index_file_path = "/content/drive/My Drive/IRE/InvertedIndexFiles/"
  query_file_path = '/content/drive/My Drive/IRE/2019201032_queries1.txt'
  output_file_path = '/content/drive/My Drive/IRE/serachtest/output.txt'
  id_title_file_path = '/content/drive/My Drive/IRE/IndexFiles/id_title.txt'

  # create title id map from file title id file
  process_title_file(id_title_file_path)
  # print(total_docs)
  # print(id_title[str(1)])

  #read words from secondary index file
  secondary_index_file_path = '/content/drive/My Drive/IRE/InvertedIndexFiles/secondaryfile.txt'
  read_secondary_index(secondary_index_file_path)

  queries = open(query_file_path, 'r').readlines()
  search_queries()
  
  # flush output to the file
  # print("time", stop-start)
  with open(output_file_path, 'w') as of:
    for line in outputs:
      of.write(line.strip()+'\n')

  

  



5, t:catcher in the rye i:axl rose b:madagascar c:2008 albums

32.453523965001295 6.490704793000259

20, Gandhi 1982

3.8077325589983957 0.19038662794991978

10, Blade Runner

4.368284473999665 0.4368284473999665

10, b:William Owen c:English songwriters

15.136559353000848 1.513655935300085

20, t:Virgin Islands r:geonames c:islands

10.381706422000207 0.5190853211000104

10, b:Vendetta c:Films about fascism

4.8466310759995395 0.48466310759995396

20, t:Graveyard i:Goro Kishitani Shigenori Takechi

9.610528437999164 0.4805264218999582

5, t:Commonwealth Games i:melbourne b:Stolenwealth c:sports

15.871963655999934 3.174392731199987

20, b:Californication Season

4.630024422998758 0.23150122114993793

10, lesbouefs somme

3.2032462209990626 0.32032462209990625

10, b:Redmond i:Los Angeles Kings c:1965 births

17.675101922000977 1.7675101922000978

10, b:Dennis Lewiston c:1934 births

9.297597004999261 0.9297597004999261

10, engel's law

7.614589659000558 0.7614589659000558

5, b:Big 